<a href="https://colab.research.google.com/github/gregwalla/Data-Challenge-Booster/blob/master/airChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#additional requirements 
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension
!pip install shap
!pip install pystan==2.19.1.1
!pip install prophet

     |████████████████████████████████| 69.2MB 43kB/s 
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK
     |████████████████████████████████| 358kB 5.1MB/s 
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491627 sha256=4421c96864f81a8a4c8c47e66228e78cd110442726cefd847096472446652d14
  Stored in directory: /root/.cache/pip/wheels/15/27/f5/a8ab9da52fd159aae6477b5ede6eaaec69fd130fa0fa59f283
Successfully built shap
     |████████████████████████████████| 71kB 3.9MB/s 
     |████████████████████████████████| 51kB 5.4MB/s 
     |████████████████████████████████| 184kB 10.2MB/s 
  Created wheel for prophet: filename=prophet-1.0.1-cp37-none-any.whl size=6639805 sha256=2055e2a7475b7d6f0eec032539d9e5b1d53f3b2ad66e28ec28ccc0bb91819b87
  Stored in directory: /root/.cache/pip/wheels/92/1d/02/bc0a91bd3a84e060722075d2b0b68e875e37ec7dc49451d513
Successfully built prophet
ERROR: fbprophet 0.7.1 has requirement cmdstanpy==0.9.5, but you

In [2]:

import pandas as pd
pd.set_option('display.max_rows', 700)
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
from pandas.tseries.offsets import DateOffset

#for plotting winds
#from windrose import WindroseAxes
#import matplotlib.cm as cm
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train_phase_1.csv')
data.date = pd.to_datetime(data.date, format='%Y-%m-%d %H:%M:%S')

submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/test_phase_1.csv')
submit.date = pd.to_datetime(submit.date, format='%Y-%m-%d %H:%M:%S')

df = data.append(submit) #renamed to submit because it is better to use 'test' in modeling jargon
df = df.sort_values(by=['date'], ascending = False) #we need to sort with most recent time first 
df = df.reset_index()

#from pandas_profiling import ProfileReport
#prof = ProfileReport(data)
#displayHTML(prof.to_html())
submit.date.min(), submit.date.max(), data.date.min(), data.date.max()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g46WzJLTo8Lm7LAVq0zEPJ074Bd37S5MakRqubuIdUThggBSd5u0hM
Mounted at /content/drive


(Timestamp('2011-01-01 01:00:00'),
 Timestamp('2012-06-28 12:00:00'),
 Timestamp('2009-07-01 01:00:00'),
 Timestamp('2012-06-26 12:00:00'))

In [3]:
#add columns to df

ones = 12 * 4 #test sequences length
zeros = 12 * 3 #train sequences length in mixed dataset 
repeat = (len(df)-zeros) / (ones + zeros) #note : we need to remove one 36 cycle to get a round number 

assert repeat == 312

repeat = int(repeat)

#we want to add an index to apply the 36/48 time cycles of the test set to the dataset as a whole
#way backward in time
#used for data prep

lst_1 = [1 for _ in range(ones)]
lst_0 = [0 for _ in range(zeros)] 
lst = lst_1 + lst_0
lst = lst * repeat + [1 for _ in range(zeros)] 

assert len(lst) == len(df)

df['idx'] = lst


#we want to generate a lag variable for every test set
#way backward in time but forward in lags
#Used in model 


lst_1 = [48-i for i in range(ones)] #test cycles
lst_0 = [0 for i in range(zeros)] #train cycles
lst = lst_1 + lst_0
lst = lst * repeat + [48 - i for i in range(zeros)] #last cycle is test cycle thus not used because no prior train cycle

assert len(lst) == len(df)

df['lags'] = lst


#we will want to index each train/test time cycles incrementally
#way backward in time
#used for data prep

lst = []
pattern = ones + zeros
for i in range(repeat): 
  lst = lst + [i for _ in range(pattern)]

#note : The last cycle is index 312 and it has only 36 entries  
lst = lst+ [repeat for _ in range(zeros)] 
  
assert len(lst) == len(df)

df['idx_cycle'] = lst


#we cycle index in 7 slots of 12 hours for each IDX cycle
#we will use it to do data augmentation
#used in model


lst = []

for _ in range(repeat+1):
  for i in reversed(range(1,8)): 
    for _ in range(12): 
      lst.append(i)
      
lst = lst[:-ones] #we take one extra cycle (36+48) thus we remove the ones

assert len(lst) == len(df)

df['12h_cycle'] = lst 

#first timestamp should be : 2009-07-02 13:00:00
#we have 312 cycles of 36/48 hours - indexed 0 to 311 
#156 cycles are train and 156 test

#Description of new columns
#idx : indicates if it is a train or test set cycle , '0' is train '1' is test
#idx_cycle : is the index of the cycle (0, to 312), 312 is not complete
#lags is lags after first missing date (1, to 48)
#12h_cycle  is division of each cyle in periods of 12 hours (1, to 7)




In [4]:

def make_ymean(df, col_name): 
  #Creation of y_hat variable which is based on average of '0' idx values for each idx cycle
  a = len(df)  
  avgs = df.groupby(['idx', 'idx_cycle']).agg('wp1').mean().reset_index() #try median also
  avgs = avgs.rename(columns = {'wp1': col_name})
  avgs = avgs.dropna()
  avgs = avgs.pivot(index="idx_cycle", columns='idx', values=col_name).reset_index()
  avgs = avgs.rename(columns =  {avgs.columns[1]: col_name, avgs.columns[2]: 'y_control'})
  avgs = avgs[['idx_cycle', col_name]]
  
  df = df.merge(avgs, on = ['idx_cycle'], how = 'left')
  
  assert len(df) == a
  
  return df

df = make_ymean(df, 'y_mean')


In [5]:
#normalize windspeed
def normal(l):
  #Scale list items between zero and one
  a = min(l)
  b = max(l)-min(l)
  if b == 0 :
    return l
  else :
    normalized = [(i- a)/b for i in l ]
    return normalized

df['ws_norm'] = normal(df['ws'])


In [6]:
#apply future to the past
df['y_mean_future'] = df['y_mean'].shift(36)
df['y_mean_future'] = df['y_mean_future'].fillna(df['ws_norm'])


In [7]:

#creating bins for magnitudes and directions (directions in my dataset are in degrees, I am converting them to direction abbreviations like N, NNW, NNE, S etc

#bins_mag= [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
#bins_mag_labels = ['0.0-0.1','0.1-0.2','0.2-0.3','0.3-0.4','0.4-0.5', '0.5-0.6']

bins_dir = [0, 11.25, 33.75, 56.25, 78.75,101.25,123.75,146.25,168.75,191.25,213.75,236.25,258.75,281.25,303.75,326.25,348.75, 360.00]
bins_dir_labels = ['N','NNE','NE','ENE','E','ESE','SE','SSE','S','SSW','SW','WSW','W','WNW','NW','NNW','North']

bins_dir_light = [0, 90, 270, 360]
bins_dir_labels_light = [1, 2, 3]


#df['mag_binned'] = pd.cut(df['magnitude(m/s)'], bins_mag, labels=bins_mag_labels)
df['dir_binned'] = pd.cut(df['wd'],bins_dir, labels=bins_dir_labels)
df['dir_binned_light'] = pd.cut(df['wd'],bins_dir_light, labels=bins_dir_labels_light)

##

Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
seasons = [('Winter', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('Spring', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('Summer', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('Autumn', (date(Y,  9, 23),  date(Y, 12, 20))),
           ('Winter', (date(Y, 12, 21),  date(Y, 12, 31)))]

def get_season(now):
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

df['season'] = df['date'].apply(get_season)


bins = [0, 5, 13, 17, 25]
labels = ['Morning','Afternoon','Evening','Night']
hours = df['date'].dt.hour
df['timeofday'] = pd.cut(hours-5+24 *(hours<5),bins=bins,labels=labels,right=False)


bins = [0, 12, 24]
labels = [0,1]
hours = df['date'].dt.hour
df['sunpower'] = pd.cut(hours,bins=bins,labels=labels,right=False)

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['hour'] = df['date'].dt.hour


In [8]:
df.head()

,index,date,wp1,u,v,ws,wd,idx,lags,idx_cycle,12h_cycle,y_mean,ws_norm,y_mean_future,dir_binned,dir_binned_light,season,timeofday,sunpower,year,month,hour
0,7487,2012-06-28 12:00:00,NaN,0.53,1.48,1.58,19.68,1,48,0,7,0.3835,0.104611,0.104611,NNE,1,Summer,Afternoon,1,2012,6,12
1,7486,2012-06-28 11:00:00,NaN,1.07,1.58,1.91,34.05,1,47,0,7,0.3835,0.127323,0.127323,NE,1,Summer,Afternoon,0,2012,6,11
2,7485,2012-06-28 10:00:00,NaN,1.63,1.61,2.29,45.42,1,46,0,7,0.3835,0.153476,0.153476,NE,1,Summer,Afternoon,0,2012,6,10
3,7484,2012-06-28 09:00:00,NaN,2.11,1.46,2.57,55.39,1,45,0,7,0.3835,0.172746,0.172746,NE,1,Summer,Morning,0,2012,6,9
4,7483,2012-06-28 08:00:00,NaN,2.44,1.08,2.67,66.14,1,44,0,7,0.3835,0.179628,0.179628,ENE,1,Summer,Morning,0,2012,6,8


In [9]:
from sklearn.model_selection import train_test_split

num_features = ['ws', 'u', 'v'] + ['y_mean', 'y_mean_future', 'ws_norm', 'lags', '12h_cycle']
cat_features = ['dir_binned', 'season', 'timeofday' , 'year', 'month' , 'hour']

for col in cat_features: 
  df[col] = df[col].astype('category')

y_col = ['wp1']
X_cols = num_features + cat_features

#idx splits the 36/48 h cycles and train set runs from 0 to 155
criteria = (df['idx_cycle'] < 156) & (df['idx'] == 1)

train_set = df.loc[-criteria, X_cols + y_col ] #minus criteria
holdout_set = df.loc[criteria, X_cols  ]

len(holdout_set), len(train_set), len(data)

(7488, 18756, 18756)

In [10]:

#we will use it to rebuild a dataframe and visualize
train_set_dates = df.loc[- criteria, 'date'] 
holdout_set_dates = df.loc[criteria, 'date'  ]

assert (len(data) == len(train_set)) & (len(submit) == len(holdout_set))

##For catboost
categorical_features_indices = np.where(train_set.dtypes != np.float)[0]

#For cross validation
X_train_set = train_set.loc[:, X_cols]
y_train_set = train_set.loc[:, y_col].values.ravel() 


#For model fit
############
train, test = train_test_split(train_set, test_size=0.2, shuffle=False) #False ? or cross validate? 

X_train = train.loc[:, X_cols]
y_train = train.loc[:, y_col].values.ravel()

X_validation = test.loc[:, X_cols]
y_validation = test.loc[:, y_col].values.ravel()
#############


In [11]:
from catboost import CatBoostRegressor, Pool, metrics, cv

model = CatBoostRegressor(
    loss_function='MAE')



model.fit(X_train, y_train, cat_features=categorical_features_indices,
          eval_set=(X_validation, y_validation), logging_level='Silent')

In [ ]:
def print_cv_summary(cv_data):
    cv_data.head(10)

    best_value = cv_data['test-MAE-mean'].min()
    best_iter = cv_data['test-MAE-mean'].values.argmin()

    print('Best validation MAE score : {:.4f}±{:.4f} on step {}'.format(
        best_value,
        cv_data['test-MAE-std'][best_iter],
        best_iter)
    )

In [ ]:

cv_params = model.get_params()

cv_data = cv(
    Pool(X_train_set, y_train_set, cat_features=categorical_features_indices),
    cv_params,
    verbose=False,
    shuffle = False,
    #plot = True,
    type = 'TimeSeries'
)
print_cv_summary(cv_data)

In [ ]:
import shap

# Track variable importance
ex = shap.TreeExplainer(model)
shap_values = ex.shap_values(X_validation)
shap.summary_plot(shap_values, X_validation)

In [ ]:
#we  standardize predictions df
def make_predict_df(dates_list, predictions_list): 
  "Make a df from our predict and also we can filter out ouf bound values"
  df_ = pd.DataFrame({
      'date': dates_list,
      'wp1': predictions_list})
  df_.loc[df_['wp1'] < 0 , 'wp1'] = 0
  df_.loc[df_['wp1'] > 1 , 'wp1'] = 1

  return df_

#used? 
predictions_train_list = model.predict(train_set)
df_predictions_train = make_predict_df(train_set_dates, predictions_train_list)

predictions_holdout_list = model.predict(holdout_set)
df_predictions_holdout = make_predict_df(holdout_set_dates, predictions_holdout_list)


assert len(df_predictions_holdout) == len(submit)




In [ ]:
df_predictions_holdout.to_csv('/content/drive/MyDrive/Colab Notebooks/data/predict.csv', index=False, sep=';')

In [ ]:
df_.head(70)

In [ ]:
#visualisation des soumissions

df_= df_predictions_holdout.merge(df_predictions_train, on = 'date' , how = 'outer')
df_=df_.merge(df, on = 'date', how = 'outer')
df_= df_.sort_values(by='date', ascending = False)


df_ =  df_.loc[(df_['date'] > 	'2010-05-01 01:00:00')&(df_['date'] < 	'2011-06-01 01:00:00'),: ]
#df_ =  df_.loc[df_['date'] > 	'2012-06-01 01:00:00',: ]

fig = plt.figure(figsize = (25,5))
ax = fig.add_subplot(111)

ax.plot(df_['date'], df_['wp1_x'], label = 'predict_on test')
ax.plot(df_['date'], df_['wp1_y'],  label = 'predict_on_train')
ax.plot(df_['date'], df_['wp1'],  label = 'true_on_train')
ax.plot(df_['date'], df_['ws_norm'],  label = 'normed_windspeed')

ax.legend()
plt.show()

#visualisation des soumissions

df_['ecart'] = df_['wp1'] - df_['wp1_y']

fig = plt.figure(figsize = (25,5))
ax = fig.add_subplot(111)

ax.plot(df_['date'], df_['ecart'], label = 'Ecart')

ax.legend()
plt.grid(True)
plt.show()

In [ ]:
Corriger le windspeed 2 fois 24 h ? 

In [ ]:
#Times series
from fbprophet import Prophet
pd.options.mode.chained_assignment = None  # default='warn'
import statistics

df_ = df.copy()  

new_df = pd.DataFrame()
maes = []

start_range = int(df_['idx_cycle'].min())
end_range  = int(df_['idx_cycle'].max()) + 1

for i in range(156, 167):

  
  ts = df_.loc[df_['idx_cycle'] == i,['date','wp1', 'ws']].rename(columns={'date':'ds', 'wp1':'y'})
  
  ts['ws'] = normal(ts['ws'])
  #train test split
  ts_test = ts.iloc[0:ones,: ]
  ts_train= ts.iloc[ones:,: ]

  ts_train['cap'] = 1
  ts_train['floor'] = 0
  
  m = Prophet(
    daily_seasonality=True, 
    growth='logistic', 
    changepoint_prior_scale=0.01) #to tune
  m.add_regressor('ws')
  m.fit(ts_train)

  future = m.make_future_dataframe(periods=48, freq='H')
  future = future.merge(ts[['ds','ws']], on = 'ds')
  future['cap'] = 1
  future['floor'] = 0

  fcst = m.predict(future)
  fcst.loc[fcst['yhat'] < 0 , 'yhat'] = 0
  fcst.loc[fcst['yhat'] > 1 , 'yhat'] = 1
  
  #We create a dataset

  assert len(fcst['yhat']) ==  len(ts['y'])
  
  ts_eval = ts.merge(fcst.loc[:, :], on='ds')
  ts_eval = ts_eval.merge(ts_train.loc[:,['ds', 'y']], on = 'ds', suffixes = [None, 'train'], how = 'left')
  ts_eval = ts_eval.merge(ts_test.loc[:,['ds', 'y']], on = 'ds', suffixes = ['y', 'test'], how = 'left')
  ts_eval['run']=i
  
  ts_eval_conso = new_df.append(ts_eval)
  

In [ ]:

#MAE
from sklearn.metrics import mean_absolute_error
y_true = ts_eval.loc[-ts_eval['ytest'].isna(), 'ytest']
y_pred = ts_eval.loc[-ts_eval['ytest'].isna(), 'yhat']

try:
  mae = mean_absolute_error(y_true, y_pred)
  print(f"Dates- train:  {ts_train.ds.min()} - {ts_train.ds.max()}, , test: {ts_test.ds.min()} - {ts_test.ds.max()}")

  msg = f"set: {i}, Mae on test: {mae:.2f}, Cumul Mae: {round(statistics.mean(maes), 2)} "
  print(msg)
  print(f"trying mae: {mae}, true {lent(y_true)}, pred {len(y_pred)}")
  maes.append(round(mae, 2))
  

except: 
  msg = f"set: {i}, Mae : Not possible"
  print(f"trying mae: {mae}, true {lent(y_true)}, pred {len(y_pred)}")
  print(msg)
    


In [ ]:
#new_df.sample(23)

x = new_df['ds']
y_cols= ['yy', 'trend', 'additive_terms', 'daily', 
         'extra_regressors_additive', 'ws_y', 'ws_x', 
         'multiplicative_terms', 'yhat' , 'ytrain', 'ytest' ]

components = ['daily', 'additive_terms', 'extra_regressors_additive', 'multiplicative_terms', 'trend']
result = ['yy', 'ws_x', 'ws_y', 'yhat', 'ytrain', 'ytest']

fig = plt.figure(figsize = (20,5))
ax = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

for col in result: 
  ax.plot(x, new_df[col], label = col)
for col in components:
  ax2.plot(x, new_df[col], label = col)

ax.legend()
ax2.legend()
plt.show()


In [ ]:
new_df.columns

In [ ]:

 run_range = 156

#visualisation des soumissions

df_visu_1 = new_df.copy()
df_visu_1['set'] = 'Predicted'

#ids splits the 36/48 h cycles and train set runs from 0 to 155
visu_criteria = (df_visu_1['run'].isin([156, 157] )

df_visu_0 = df.loc[visu_criteria, ['date','wp1']]
df_visu_0['set'] = 'Train'

df_plot = df_visu_0.append(df_visu_1).sort_values(by = 'date')

df_plot_filtered =  df_plot.loc[df_plot['date'] > 	'2012-03-28 08:00:00',: ]

fig = plt.figure(figsize = (20,5))
ax = fig.add_subplot(111)

df_plot_filtered =  df_plot.loc[df_plot['date'] > 	'2012-03-28 08:00:00',: ]

Predicted = df_plot_filtered.loc[df_plot_filtered['set'] == 'Predicted', ['date','wp1']]
Train = df_plot_filtered.loc[-(df_plot_filtered['set'] == 'Predicted'), ['date','wp1']]

ax.plot(Predicted['date'], Predicted['wp1'], label = 'Predicted')
ax.plot(Train['date'], Train['wp1'],  label = 'Train')
ax.legend()
plt.show()

In [ ]:

#Data augmentation
#we want to append the train set with itself but we will be shifting lag and 12h cycles indices
#12h cycle in conjunction with lags enable to preserve unicity of rows
#there will be however one 12h overlap , maybe we should filter it out 

#cols_to_shift = ['12h_cycle', 'idx', 'lags', 'idx_cycle']
#shifts = 36

#df2 = df.copy()
#for col in cols_to_shift: 
#  df2[col] = df2[col].shift(shifts)
  
#df2 = make_yhat(df2)
#df2['augment_set'] = 1


#clean cut 
#assert len(df2.loc[df2['idx_cycle'] >= 156, :]) == len(df2.loc[df2['idx_cycle'] < 156, :])

#we cut the 36 earliest dates because the last shift is not complete
#df = df.loc[df['date'] >'2009-07-02 12:00:00', :]


#df['augment_set'] = 0

#assert len(df.loc[df['idx_cycle'] >= 156, :]) == len(df.loc[df['idx_cycle'] < 156, :])

###############
###FOR Prophet###
#we will save an instance of the dataset to use prophet for later

#df_ts2 = df2.loc[df2['idx_cycle'] >= 156 ,:]
###############
###############

#test set remains the same but train set is 2 x bigger 
#df = df.append(df2.loc[df2['idx_cycle'] >= 156, :]) #we only augment train set


#########Next steps
#prophet 

#to simplify use cycles before 156 with both data and test


#for each 156*2 cycle : 
#predict last 36 with a simple prophet 

#isolate a time seties of 36 starting from the first, exluding the oldest 36 
#take the 
#predict wind power for 36 based on wind speed regressor 
#
#predict wind
#for each 155 cycle :
#reverse 
#create reversed cycle 
#except for last predict where we can take WS
#isolate a time seties of 36+24 , 24 belonging to a evaluation set, 36 belonging to future
#inverse the wp column, so the last 24 rows belong to the previous evaluation set (they sould be empty on validation)
#take 36 predict 24 with wind as regressor
#inverse a new the wp column  

#créer winds_speed coor sur les 24 premiers créneaux


#take 36 predict 24 with wind as regressor




In [ ]:
#create windspeed 
#augment data at 00h and 12h cutoffs 
#